# సెషన్ 5 – బహుళ ఏజెంట్ ఆర్కెస్ట్రేటర్

Foundry Local ఉపయోగించి ఒక సాదారణ రెండు-ఏజెంట్ పైప్‌లైన్ (శోధకుడు -> ఎడిటర్) ను ప్రదర్శిస్తుంది.


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్  
స్థానిక మోడల్ యాక్సెస్ మరియు చాట్ కంప్లీషన్స్ కోసం అవసరమైన `foundry-local-sdk` మరియు `openai` ను ఇన్‌స్టాల్ చేస్తుంది. ఐడంపొటెంట్.


# సన్నివేశం
కనిష్ట రెండు-ఏజెంట్ ఆర్కెస్ట్రేటర్ నమూనాను అమలు చేస్తుంది:
- **శోధన ఏజెంట్** సంక్షిప్త వాస్తవ బుల్లెట్లు సేకరిస్తుంది
- **ఎడిటర్ ఏజెంట్** కార్యనిర్వాహక స్పష్టత కోసం పునఃరచన చేస్తుంది

ప్రతి ఏజెంట్‌కు పంచుకున్న మెమరీ, మధ్యవర్తి అవుట్పుట్‌ను వరుసగా పంపడం, మరియు ఒక సరళమైన పైప్‌లైన్ ఫంక్షన్‌ను ప్రదర్శిస్తుంది. మరిన్ని పాత్రలకు (ఉదా: విమర్శకుడు, ధృవీకర్త) లేదా సమాంతర శాఖలకు విస్తరించవచ్చు.

**పరిసర వేరియబుల్స్:**
- `FOUNDRY_LOCAL_ALIAS` - ఉపయోగించాల్సిన డిఫాల్ట్ మోడల్ (డిఫాల్ట్: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - ప్రాథమిక ఏజెంట్ మోడల్ (ALIAS ను మించిపోతుంది)
- `AGENT_MODEL_EDITOR` - ఎడిటర్ ఏజెంట్ మోడల్ (ప్రాథమిక మోడల్‌కు డిఫాల్ట్)

**SDK సూచన:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**ఇది ఎలా పనిచేస్తుంది:**
1. **FoundryLocalManager** Foundry Local సేవను ఆటోమేటిక్‌గా ప్రారంభిస్తుంది
2. పేర్కొన్న మోడల్‌ను డౌన్లోడ్ చేసి లోడ్ చేస్తుంది (లేదా క్యాష్ చేసిన వెర్షన్ ఉపయోగిస్తుంది)
3. పరస్పర చర్య కోసం OpenAI-అనుకూల ఎండ్‌పాయింట్‌ను అందిస్తుంది
4. ప్రతి ఏజెంట్ ప్రత్యేక పనుల కోసం వేరే మోడల్‌ను ఉపయోగించవచ్చు
5. బిల్ట్-ఇన్ రీట్రై లాజిక్ తాత్కాలిక వైఫల్యాలను సులభంగా నిర్వహిస్తుంది

**ప్రధాన లక్షణాలు:**
- ✅ ఆటోమేటిక్ సేవా కనుగొనడం మరియు ప్రారంభం
- ✅ మోడల్ జీవనచక్ర నిర్వహణ (డౌన్లోడ్, క్యాష్, లోడ్)
- ✅ పరిచిత API కోసం OpenAI SDK అనుకూలత
- ✅ ఏజెంట్ ప్రత్యేకత కోసం బహుముఖ మోడల్ మద్దతు
- ✅ రీట్రై లాజిక్‌తో బలమైన లోప నిర్వహణ
- ✅ స్థానిక ఇన్ఫరెన్స్ (క్లౌడ్ API అవసరం లేదు)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### వివరణ: కోర్ ఇంపోర్ట్స్ & టైపింగ్
ఏజెంట్ సందేశ నిల్వ కోసం డాటాక్లాసులను మరియు స్పష్టత కోసం టైపింగ్ సూచనలను పరిచయం చేస్తుంది. తరువాతి ఏజెంట్ చర్యల కోసం Foundry Local మేనేజర్ + OpenAI క్లయింట్‌ను ఇంపోర్ట్ చేస్తుంది.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### వివరణ: మోడల్ ప్రారంభం (SDK నమూనా)
దృఢమైన మోడల్ నిర్వహణ కోసం Foundry Local Python SDK ఉపయోగిస్తుంది:
- **FoundryLocalManager(alias)** - సేవను ఆటోమేటిక్‌గా ప్రారంభించి అలియాస్ ద్వారా మోడల్‌ను లోడ్ చేస్తుంది
- **get_model_info(alias)** - అలియాస్‌ను నిర్దిష్ట మోడల్ ID గా పరిష్కరిస్తుంది
- **manager.endpoint** - OpenAI క్లయింట్ కోసం సేవా ఎండ్‌పాయింట్‌ను అందిస్తుంది
- **manager.api_key** - API కీని అందిస్తుంది (స్థానిక ఉపయోగానికి ఐచ్ఛికం)
- వేర్వేరు ఏజెంట్ల కోసం వేర్వేరు మోడల్స్‌ను మద్దతు ఇస్తుంది (ప్రధాన మరియు ఎడిటర్)
- సహనానికి ఎక్స్‌పోనెన్షియల్ బ్యాక్‌ఆఫ్‌తో బిల్ట్-ఇన్ రీట్రై లాజిక్
- సేవ సిద్ధంగా ఉందని నిర్ధారించడానికి కనెక్షన్ ధృవీకరణ

**ప్రధాన SDK నమూనా:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**జీవన చక్ర నిర్వహణ:**
- మేనేజర్లు సరైన శుభ్రపరిచేందుకు గ్లోబల్‌గా నిల్వ చేయబడతారు
- ప్రతి ఏజెంట్ ప్రత్యేకత కోసం వేర్వేరు మోడల్‌ను ఉపయోగించవచ్చు
- ఆటోమేటిక్ సేవా కనుగొనడం మరియు కనెక్షన్ నిర్వహణ
- విఫలమైతే ఎక్స్‌పోనెన్షియల్ బ్యాక్‌ఆఫ్‌తో సున్నితమైన రీట్రై

ఇది ఏజెంట్ ఆర్కెస్ట్రేషన్ ప్రారంభం కంటే ముందు సరైన ప్రారంభాన్ని నిర్ధారిస్తుంది.

**సూచన:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### వివరణ: ఏజెంట్ & మెమరీ తరగతులు
మెమరీ ఎంట్రీల కోసం తేలికపాటి `AgentMsg` మరియు కిందివి కలిగిన `Agent` ను నిర్వచిస్తుంది:
- **సిస్టమ్ పాత్ర** - ఏజెంట్ వ్యక్తిత్వం మరియు సూచనలు
- **సందేశ చరిత్ర** - సంభాషణ సందర్భాన్ని నిర్వహిస్తుంది
- **act() పద్ధతి** - సరైన లోప నిర్వహణతో చర్యలను అమలు చేస్తుంది

ఏజెంట్ వివిధ మోడల్స్ (ప్రధాన మరియు ఎడిటర్) ఉపయోగించవచ్చు మరియు ప్రతి ఏజెంట్‌కు వేరుగా సందర్భాన్ని నిర్వహిస్తుంది. ఈ నమూనా వీటిని సాధ్యమవుతుంది:
- చర్యల మధ్య మెమరీ నిల్వ
- ప్రతి ఏజెంట్‌కు అనుకూలమైన మోడల్ కేటాయింపు
- లోపాల వేరుచేయడం మరియు పునరుద్ధరణ
- సులభమైన చైన్ మరియు సమన్వయం


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### వివరణ: సమన్వయ పద్ధతి
రెండు ప్రత్యేక ఏజెంట్లను సృష్టిస్తుంది:
- **శోధకుడు**: ప్రాథమిక మోడల్ ఉపయోగించి, వాస్తవ సమాచారాన్ని సేకరిస్తాడు
- **సంపాదకుడు**: వేరే మోడల్ ఉపయోగించవచ్చు (కాన్ఫిగర్ చేసినట్లయితే), మెరుగుపరచి తిరిగి రాస్తాడు

`pipeline` ఫంక్షన్:
1. శోధకుడు ముడి సమాచారాన్ని సేకరిస్తాడు
2. సంపాదకుడు కార్యనిర్వాహక-సిద్ధమైన అవుట్పుట్‌గా మెరుగుపరుస్తాడు
3. మధ్యంతర మరియు తుది ఫలితాలను రెండింటినీ తిరిగి ఇస్తుంది

ఈ నమూనా అనుమతిస్తుంది:
- మోడల్ ప్రత్యేకత (విభిన్న పాత్రల కోసం వేరే మోడల్స్)
- బహుళ దశల ప్రాసెసింగ్ ద్వారా నాణ్యత మెరుగుదల
- సమాచార మార్పిడి ట్రేసబిలిటీ
- మరిన్ని ఏజెంట్లు లేదా సమాంతర ప్రాసెసింగ్‌కు సులభమైన విస్తరణ


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### వివరణ: పైప్‌లైన్ అమలు & ఫలితాలు
కంప్లయెన్స్ + లేటెన్సీ థీమ్ ఉన్న ప్రశ్నపై బహుళ ఏజెంట్ పైప్‌లైన్‌ను అమలు చేస్తుంది, దీని ద్వారా చూపించబడుతుంది:
- బహుళ దశల సమాచార మార్పిడి
- ఏజెంట్ ప్రత్యేకత మరియు సహకారం
- శుద్ధి ద్వారా అవుట్పుట్ నాణ్యత మెరుగుదల
- ట్రేసబిలిటీ (మధ్యంతర మరియు తుది అవుట్పుట్లు రెండూ నిల్వ చేయబడ్డాయి)

**ఫలిత నిర్మాణం:**
- `question` - అసలు వినియోగదారు ప్రశ్న
- `research` - ముడి పరిశోధన అవుట్పుట్ (నిజమైన బుల్లెట్లు)
- `final` - శుద్ధి చేసిన కార్యనిర్వాహక సారాంశం
- `models` - ప్రతి దశకు ఉపయోగించిన మోడల్స్

**విస్తరణ ఆలోచనలు:**
1. నాణ్యత సమీక్ష కోసం క్రిటిక్ ఏజెంట్‌ను జోడించండి
2. వేర్వేరు అంశాల కోసం సమాంతర పరిశోధన ఏజెంట్లను అమలు చేయండి
3. నిజ నిర్ధారణ కోసం వెరిఫైయర్ ఏజెంట్‌ను జోడించండి
4. వేర్వేరు సంక్లిష్టత స్థాయిల కోసం వేర్వేరు మోడల్స్ ఉపయోగించండి
5. పునరావృత మెరుగుదల కోసం ఫీడ్‌బ్యాక్ లూప్‌లను అమలు చేయండి


### అధునాతన: కస్టమ్ ఏజెంట్ కాన్ఫిగరేషన్

ప్రారంభ సెల్‌ను నడిపించే ముందు వాతావరణ చరాలు మార్చడం ద్వారా ఏజెంట్ ప్రవర్తనను అనుకూలీకరించండి:

**లభ్యమయ్యే మోడల్స్:**
- అందుబాటులో ఉన్న అన్ని మోడల్స్‌ను చూడటానికి టెర్మినల్‌లో `foundry model ls` ఉపయోగించండి
- ఉదాహరణలు: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, మొదలైనవి.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
